In [4]:
import random
import math
import pandas as pd

In [61]:
%%html
<img src='neural_network.jpg', width=500, height=300>

In [51]:
class Neuron:
    
    def __init__(self, bias):
        self.bias = bias
        self.weights = []
        
    # Apply the logistic function (sigmoid) to activation_function the output of the neuron
    def activation_function(self):#, inputs, weights):
        return 1/(1 + math.exp(-(np.dot(self.inputs, self.weights) + self.bias)))
    
    def calculate_output(self, inputs):
        self.inputs = inputs
        #self.weights = weights
        self.output = self.activation_function()
        return self.output
                  
    # The error for each neuron is calculated by the Mean Square Error method:
    def calculate_error(self, target_output):
        return 0.5*(target_output - self.output) ** 2
                  
    # The partial derivate of the error with respect to actual output then is calculated by:
    # = 2 * 0.5 * (target output - actual output) ^ (2 - 1) * -1
    # = ∂E/∂yⱼ = -(tⱼ - yⱼ)
    def calculate_partial_derivative_E_yj(self, target_output):
        return -(target_output - self.output)
    
    # The total net input into the neuron is activation_functioned using logistic function to calculate the neuron's output:
    # The derivative (not partial derivative since there is only one variable) of the output then is:
    # dyⱼ/dzⱼ = yⱼ * (1 - yⱼ) -- the derivative of sigmoid function
    def calculate_partial_derivative_Yj_zj(self):
        return self.output*(1 - self.output)
                  
    # the partial derivative of the error with respect to the total net input.
    # This value is also known as the delta (δ) [1]
    # δ = ∂E/∂zⱼ = ∂E/∂yⱼ * ∂yⱼ/∂zⱼ 
    def calculate_partial_derivative_E_zj(self, target_output):
        return self.calculate_partial_derivative_E_yj(target_output)*self.calculate_partial_derivative_Yj_zj()

    # The total net input is the weighted sum of all the inputs to the neuron and their respective weights:
    # = zⱼ = netⱼ = x₁w₁ + x₂w₂ ...
    # The partial derivative of the total net input with respective to a given weight (with everything else held constant) then is:
    def calculate_partial_derivative_Zj_wi(self, index):
        return self.inputs[index]
    
    # Calculate derivative of Yj to wi
    def calculate_partial_derivative_Yj_wi(self, index):
        return self.calculate_partial_derivative_Yj_zj()*self.inputs[index]

In [41]:
class NeuronLayer:

    def __init__(self, num_neurons, bias):

        # Every neuron in a layer shares the same bias-- because it's just for initialization 
        self.bias = bias if bias else random.random()
        random.random()
        self.neurons = []
        for i in range(num_neurons):
            self.neurons.append(Neuron(self.bias))
        
    def inspect(self):
        # print out info
        print('Neurons:', len(self.neurons))
        for n in range(len(self.neurons)):
            print(' Neuron', n)
            for w in range(len(self.neurons[n].weights)):
                print('  Weight:', self.neurons[n].weights[w])
            print('  Bias:', self.bias)

    def feed_forward(self, inputs):
        # calculate using inputs and weights
        outputs = []
        for neuron in self.neurons:
            outputs.append(neuron.calculate_output(inputs))
        return outputs

    def get_outputs(self):
        outputs = []
        for neuron in self.neurons:
            outputs.append(neuron.output)
        return outputs

In [56]:
class NeuralNetwork:
    
    def __init__(self, num_inputs, num_hidden, num_outputs, hidden_layer_weights=None, hidden_layer_bias=None,
                 output_layer_weights=None, output_layer_bias=None, LEARNING_RATE = 0.5):
        
        # Initialization of a 3 layer network
        self.num_inputs = num_inputs
        self.hidden_layer = NeuronLayer(num_hidden, hidden_layer_bias)
        self.output_layer = NeuronLayer(num_outputs, output_layer_bias)
        self.init_weights_from_inputs_to_hidden_layer_neurons(hidden_layer_weights)
        self.init_weights_from_hidden_layer_neurons_to_output_layer_neurons(output_layer_weights)
        self.LEARNING_RATE = LEARNING_RATE

    def init_weights_from_inputs_to_hidden_layer_neurons(self, hidden_layer_weights):
        weight_num = 0
        for h in range(len(self.hidden_layer.neurons)): 
            for i in range(self.num_inputs):  
                if not hidden_layer_weights:
                    # if no values for hidden_layer_weights is input, set random values
                    self.hidden_layer.neurons[h].weights.append(random.random())
                else:
                    self.hidden_layer.neurons[h].weights.append(hidden_layer_weights[weight_num])
                weight_num += 1

    def init_weights_from_hidden_layer_neurons_to_output_layer_neurons(self, output_layer_weights):
        weight_num = 0
        for o in range(len(self.output_layer.neurons)):  
            for h in range(len(self.hidden_layer.neurons)): 
                if not output_layer_weights:
                    self.output_layer.neurons[o].weights.append(random.random())
                else:
                    self.output_layer.neurons[o].weights.append(output_layer_weights[weight_num])
                weight_num += 1

    def prinT_NN(self):  # output info for NN
        print('------')
        print('* Inputs: {}'.format(self.num_inputs))
        print('------')
        print('Hidden Layer')
        self.hidden_layer.prinT_NN()
        print('------')
        print('* Output Layer')
        self.output_layer.prinT_NN()
        print('------')

    def feed_forward(self, inputs): 
        hidden_layer_outputs = self.hidden_layer.feed_forward(inputs) #inputs to hidden
        return self.output_layer.feed_forward(hidden_layer_outputs) #hidden to outputs

    # Uses online learning to train NN
    def train(self, training_inputs, training_outputs):
        self.feed_forward(training_inputs)
        
        # 1. Output neuron deltas 
        partial_derivative_output_neuron_E_zj = [0]*len(self.output_layer.neurons)
        for o in range(len(self.output_layer.neurons)):
            # ∂E/∂zⱼ=∂E/∂a*∂a/∂z=cost'(target_output)*sigma'(z)
            partial_derivative_output_neuron_E_zj[o] = self.output_layer.neurons[
                o].calculate_partial_derivative_E_zj(training_outputs[o])

        # 2. Hidden neuron deltas
        derivative_hidden_neuron_E_yj = [0]*len(self.hidden_layer.neurons)
        for h in range(len(self.hidden_layer.neurons)):
            # We need to calculate the derivative of the error with respect to the output of each hidden layer neuron
            # dE/dyⱼ = Σ ∂E/∂zⱼ * ∂z/∂yⱼ = Σ ∂E/∂zⱼ * wᵢⱼ , y is hidden layer output, 
            #wij is the weight from hidden layer to output layer
            derivative_E_yj = 0
            for o in range(len(self.output_layer.neurons)):
                derivative_E_yj += partial_derivative_output_neuron_E_zj[o]* \
                                                    self.output_layer.neurons[o].weights[h]
            # ∂E/∂zⱼ = dE/dyⱼ * ∂yj/∂zj
            derivative_hidden_neuron_E_yj[h] = derivative_E_yj*self.hidden_layer.neurons[
                h].calculate_partial_derivative_Yj_zj()

        # 3. Update output neuron weights 
        for o in range(len(self.output_layer.neurons)):
            for weight_hidden_output in range(len(self.output_layer.neurons[o].weights)):
                # ∂Eⱼ/∂wᵢⱼ = ∂E/∂zⱼ * ∂zⱼ/∂wᵢⱼ
                partial_derivative_E_wi = partial_derivative_output_neuron_E_zj[o]*self.output_layer.neurons[
                    o].calculate_partial_derivative_Yj_wi(weight_hidden_output)
                # Δw = α * ∂Eⱼ/∂wᵢ
                self.output_layer.neurons[o].weights[weight_hidden_output] -= self.LEARNING_RATE*partial_derivative_E_wi

        # 4. Update hidden neuron weights 
        for h in range(len(self.hidden_layer.neurons)):
            for weight_input_hidden in range(len(self.hidden_layer.neurons[h].weights)):
                # ∂Eⱼ/∂wᵢ = ∂E/∂zⱼ * ∂zⱼ/∂wᵢ
                partial_derivative_E_wi = derivative_hidden_neuron_E_yj[h]*self.hidden_layer.neurons[
                    h].calculate_partial_derivative_Yj_wi(weight_input_hidden)
                # Δw = α * ∂Eⱼ/∂wᵢ
                self.hidden_layer.neurons[h].weights[weight_input_hidden] -= self.LEARNING_RATE*partial_derivative_E_wi

    def calculate_total_error(self, training_sets):
        total_error = 0
        for t in range(len(training_sets)):
            training_inputs, training_outputs = training_sets[t]
            self.feed_forward(training_inputs)
            total_error = sum(self.output_layer.neurons[o].calculate_error(training_outputs[o]) for o in range(len(training_outputs)))
        return total_error

In [65]:
nn = NeuralNetwork(2, 2, 2, hidden_layer_weights=[0.15, 0.2, 0.25, 0.3], hidden_layer_bias=0.35,
                   output_layer_weights=[0.4, 0.45, 0.5, 0.55], output_layer_bias=0.6)
for i in range(1000):
    nn.train([0.8, 0.2], [0.01, 0.99])
    print(i, round(nn.calculate_total_error([[[0.05, 0.1], [0.01, 0.99]]]), 9)) 

0 0.296959421
1 0.295535007
2 0.294097817
3 0.292647808
4 0.291184945
5 0.2897092
6 0.288220552
7 0.286718989
8 0.285204505
9 0.283677107
10 0.282136807
11 0.28058363
12 0.279017608
13 0.277438786
14 0.275847218
15 0.27424297
16 0.272626119
17 0.270996754
18 0.269354976
19 0.267700901
20 0.266034654
21 0.264356378
22 0.262666226
23 0.260964366
24 0.259250982
25 0.257526272
26 0.255790447
27 0.254043737
28 0.252286383
29 0.250518646
30 0.248740799
31 0.246953134
32 0.245155957
33 0.243349591
34 0.241534375
35 0.239710665
36 0.237878831
37 0.236039261
38 0.234192357
39 0.232338538
40 0.230478237
41 0.228611904
42 0.226740002
43 0.224863008
44 0.222981413
45 0.221095723
46 0.219206453
47 0.217314134
48 0.215419304
49 0.213522516
50 0.211624328
51 0.209725312
52 0.207826043
53 0.205927106
54 0.20402909
55 0.202132592
56 0.20023821
57 0.198346546
58 0.196458204
59 0.194573787
60 0.1926939
61 0.190819144
62 0.188950117
63 0.187087416
64 0.185231629
65 0.183383339
66 0.181543123
67 0.17971154

926 0.01862857
927 0.018616889
928 0.018605231
929 0.018593596
930 0.018581983
931 0.018570392
932 0.018558824
933 0.018547278
934 0.018535754
935 0.018524253
936 0.018512774
937 0.018501316
938 0.018489881
939 0.018478468
940 0.018467076
941 0.018455707
942 0.018444359
943 0.018433032
944 0.018421727
945 0.018410444
946 0.018399182
947 0.018387942
948 0.018376723
949 0.018365525
950 0.018354348
951 0.018343192
952 0.018332058
953 0.018320944
954 0.018309851
955 0.018298779
956 0.018287728
957 0.018276698
958 0.018265688
959 0.018254699
960 0.018243731
961 0.018232782
962 0.018221855
963 0.018210947
964 0.01820006
965 0.018189193
966 0.018178347
967 0.01816752
968 0.018156714
969 0.018145927
970 0.01813516
971 0.018124414
972 0.018113687
973 0.018102979
974 0.018092292
975 0.018081624
976 0.018070975
977 0.018060347
978 0.018049737
979 0.018039147
980 0.018028576
981 0.018018025
982 0.018007493
983 0.01799698
984 0.017986486
985 0.017976011
986 0.017965555
987 0.017955118
988 0.0179446